# The Movies Dataset TF-IDF Content-Based Recommendation

##Понимание данных:

### Набор данных

Дан набор данных, содержащий информацию о 45,000 фильмах, выпущенных до июля 2017 года. Набор данных представлен на ресурсе Kaggle по ссылке https://www.kaggle.com/rounakbanik/the-movies-dataset
где представлено следующее описание составляющих файлов (выполнен перевод на русский язык):

`movies_metadata.csv:` Основной файл метаданных фильмов. Содержит информацию о 45 000 фильмов, представленных в наборе данных Full MovieLens. В таблицы представлены плакаты, фоны, бюджет, доход, даты выпуска, языки, страны-производители и компании.

`keywords.csv:` Содержит ключевые слова сюжета для наших фильмов MovieLens. Доступен в виде строкового объекта JSON.

`credits.csv:` Состоит из информации об актерах и съемках всех наших фильмов. Доступен в виде строкового объекта JSON.

`links.csv:` Файл содержит идентификаторы TMDB и IMDB всех фильмов, представленных в наборе данных Full MovieLens.

`links_small.csv:` Содержит идентификаторы TMDB и IMDB небольшого подмножества из 9000 фильмов полного набора данных.

`ratings_small.csv:` Подмножество 100 000 оценок от 700 пользователей на 9 000 фильмов.

`ratings.csv `- файл, содержащий полный список оценок, выставленных пользователями фильмам



Рассмотрим подробнее две таблицы:

*   `movies_metadata.csv `
*   `ratings.csv `

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
! pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown 1VhTKhVGzX5yzhWUIS_6BxueDCMdY6qMs

Downloading...
From: https://drive.google.com/uc?id=1VhTKhVGzX5yzhWUIS_6BxueDCMdY6qMs
To: /content/movies_metadata.csv
100% 34.4M/34.4M [00:00<00:00, 118MB/s] 


In [ ]:
!gdown 1isn48f2xVc-XEdvSrBgqc94r-_NXqtGd

Downloading...
From: https://drive.google.com/uc?id=1isn48f2xVc-XEdvSrBgqc94r-_NXqtGd
To: /content/ratings.csv
100% 710M/710M [00:07<00:00, 92.7MB/s]


Начинаем с анализа датафрейма movies_metadata

In [ ]:
metadata = pd.read_csv("movies_metadata.csv", low_memory=False)
print(f'Всего строк: {len(metadata)}')

Всего строк: 45466


## Задание 1

In [ ]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Удалите из датафрейма metadata строки, в которых отсутствует описание. Обратите внимание, что у некоторых фильмов формально описание есть, но там написано No overview found, No overview, No movie overview available, Released. Такие строки тоже нужно удалить.

In [ ]:
metadata_new = metadata.loc[~pd.isna(metadata.overview)]

Удалим строки с описанием из стоп-листа

In [ ]:
drop_list = ['No overview found.', 'No overview yet.', 'No overview.', 'No overview', 'No movie overview available.', 'No plot overview available', 'No movie overview available, please add one at themoviedb.org',
             'no overview yet', 'Released', '...', ' ', 'x']
metadata = metadata_new.loc[~metadata.overview.isin(drop_list)]

Осталось строк:

In [ ]:
print(f'Всего строк: {len(metadata)}')

Всего строк: 44359


In [ ]:
metadata.isnull().sum()

adult                        0
belongs_to_collection    39945
budget                       0
genres                       0
homepage                 36606
id                           0
imdb_id                     15
original_language           10
original_title               0
overview                     0
popularity                   3
poster_path                344
production_companies         3
production_countries         3
release_date                74
revenue                      3
runtime                      3
spoken_languages             3
status                      66
tagline                  23961
title                        3
video                        3
vote_average                 3
vote_count                   3
dtype: int64

## Задание 2

Оставьте в датафрейме столбцы `'id', 'imdb_id', 'overview', 'title'`. Выведите 10 случайных строк.

In [ ]:
metadata = metadata[['id', 'imdb_id', 'overview', 'title']]
metadata.sample(10)

,id,imdb_id,overview,title
20730,173294,tt2006810,Following his ruin in the latest banking crisi...,Papadopoulos & Sons
18900,242621,tt0037946,A Barbary Coast saloon owner hopes to marry hi...,Nob Hill
7596,25853,tt0285728,"On February 15, 1992 in Milwaukee, Wisconsin, ...",Dahmer
23453,32929,tt0042219,"When he's discharged from a military hospital,...",Backfire
8812,13633,tt0085210,Mick O'Brien is a young Chicago street thug to...,Bad Boys
14607,23963,tt1226681,When disc jockey Grant Mazzy reports to his ba...,Pontypool
25579,60119,tt0244196,"A tough guy named Tommy Gunn, with ""kaos"" tatt...",Second Skin
20205,135200,tt1853555,"Jennifer Carpenter, Kristen Connolly and Alexa...",Ex-Girlfriends
35915,190183,tt0031648,A confidence man pretending to be a mentalist ...,Midnight Shadow
12823,19244,tt1086340,The true-life story of a Harlem's notorious Ni...,Mr. Untouchable


## Задание 3

Загрузим рейтинги

In [ ]:
ratings = pd.read_csv("ratings.csv", low_memory=False)

Посчитаем строки

In [ ]:
print(f'Всего строк: {len(ratings)}')

Всего строк: 26024289


Выведем 10 случайных

In [ ]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148
6,1,2762,4.5,1425941300
7,1,2918,5.0,1425941593
8,1,2959,4.0,1425941601
9,1,4226,4.0,1425942228


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


Убедимся в отсутствии пропусков

In [ ]:
pd.isnull(ratings).sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Объедините датафреймы `metadata` и `ratings` в один. Обратите внимание, что `'id'` в  `metadata`, этот тот же самый идентификатор, что и `'movie_id'` в `ratings`. Объединять нужно по этому идентификатору (также обратите внимание на его тип данных).

Приведем к нужному типу и объединим

In [ ]:
metadata['id'] = metadata['id'].astype(np.int64)

In [ ]:
meta_new = metadata.rename(columns={"id": "movieId"})
df = meta_new.merge(ratings, on='movieId')
df

,movieId,imdb_id,overview,title,userId,rating,timestamp
0,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,1923,3.0,858335006
1,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,2103,5.0,946044912
2,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,5380,1.0,878941641
3,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,6177,4.0,859415226
4,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",Toy Story,6525,4.0,857388995
...,...,...,...,...,...,...,...
11361665,111109,tt2028550,An artist struggles to finish his work while a...,Century of Birthing,33940,2.5,1405878785
11361666,111109,tt2028550,An artist struggles to finish his work while a...,Century of Birthing,172224,3.0,1399502972
11361667,111109,tt2028550,An artist struggles to finish his work while a...,Century of Birthing,210792,3.0,1467090449
11361668,111109,tt2028550,An artist struggles to finish his work while a...,Century of Birthing,225396,3.5,1399302912


## Задание 4

Появились ли пропуски в получившемся после объединения датафрейме? Если появились, то ответьте на вопрос "почему?" и удалите строки с пропусками.

In [ ]:
df.isnull().sum()

movieId       0
imdb_id      47
overview      0
title         0
userId        0
rating        0
timestamp     0
dtype: int64

Заметим, что в столбце 'imdb_id' появилось 47 пропусков. Можно предположить, что причиной этого стало наличие пропусков в исходном датасете в столбце 'imdb_id'. При процедуре объединения двух датасетов могут мерджиться строки, как раз содержащие пропуски, что и является следствием их возникновения в новом датасете. Убедимся, что в metadata действительно имеются пропуски в столбце 'imdb_id':

In [ ]:
metadata.isnull().sum()

id           0
imdb_id     15
overview     0
title        3
dtype: int64

Видно, что в искомом столбце содержится 15 пропусков для 'imdb_id'. Наше предположение подтвердилось. Удалим эти строки:

In [ ]:
print(f'Размер датафрейма до удаления: {df.shape}')
df = df[pd.notnull(df.imdb_id)]

Размер датафрейма до удаления: (11361670, 7)


Выведите размеры получившегося датафрейма и 10 случайных записей в нём.

In [ ]:
print(f'Размер датафрейма после удаления: {df.shape}')

Размер датафрейма после удаления: (11361623, 7)


In [ ]:
df.sample(10)

,movieId,imdb_id,overview,title,userId,rating,timestamp
2067844,466,tt0067309,This acclaimed thriller stars Jane Fonda as Br...,Klute,33121,2.5,1148077181
3663896,1552,tt0098067,"The story of the Buckman family and friends, a...",Parenthood,81717,1.5,1442596800
5683664,22,tt0325980,"Jack Sparrow, a freewheeling 17th-century pira...",Pirates of the Caribbean: The Curse of the Bla...,8868,3.0,840442467
8963970,1127,tt0434292,"Set in Spain, the story is about friendship an...",Princesses,130708,5.0,952798242
3299512,1213,tt0134119,Tom Ripley is a calculating young man who beli...,The Talented Mr. Ripley,268392,3.0,1424342358
1608,949,tt0113277,"Obsessive master thief, Neil McCauley leads a ...",Heat,127486,3.5,1357855534
1699729,1968,tt0119141,Alex Whitman (Matthew Perry) is a designer fro...,Fools Rush In,92676,4.5,1160286731
5752346,26246,tt0104504,"On June 26, 1975, during a period of high tens...",Incident at Oglala,45908,3.5,1262563442
6732108,435,tt0319262,After years of increases in the greenhouse eff...,The Day After Tomorrow,61471,3.0,837607776
7886592,5620,tt0065569,Jess Franco's version of the Bram Stoker class...,Count Dracula,233951,2.5,1124219790


## Задание 5

Возьмите случайного пользователя (проверьте, чтобы у него было достаточное количество оценок). Сформируйте для этого пользователя рекомендацию методом коллаборативной фильтрации. Оцените качество это рекомендации.


Повторите эти рассчеты для большого числа пользователей и дайте интегральную оценку.

Построи Коллаборативную фильтрацию по сходству, основанному на евклидовом расстоянии

In [ ]:
class Collaborative:
  def __init__(self, dataframe):
    self.dataframe = dataframe
    self.user_list = dataframe['userId'].unique()
    self.user_ratings = {userId: self.get_ratings(userId) for userId in self.user_list}
    self.movies = dict(df[['movieId', 'title']].groupby(['movieId', 'title']).count().reset_index().values.tolist())

  def get_ratings(self, userId):
    user_df = self.dataframe.query(f'userId == {userId}')
    return dict(zip(user_df['movieId'], user_df['rating']))

  def euclid_distance(self, x, y, power=2):
    distance = 0
    for x_value, y_value in zip(x, y):
        distance += (x_value - y_value) ** power
    return distance ** (1 / power)

  def similarity_distance(self, movies_dataframe):
    if len(movies_dataframe) == 0:
        return 0
    return 1 / (1 + self.euclid_distance(movies_dataframe['rating_first'], movies_dataframe['rating_second']))

  def remove_user(self, user):
        return list(set(self.user_list) - set([user]))

  def common_movies(self, first_user, second_user):
      movies_first_user = self.user_ratings[first_user]
      movies_second_user = self.user_ratings[second_user]
      return pd.merge(
          pd.DataFrame(data={'movieId': movies_first_user.keys(), 'rating': movies_first_user.values()}),
          pd.DataFrame(data={'movieId': movies_second_user.keys(), 'rating': movies_second_user.values()}),
          on='movieId',
          suffixes=['_first', '_second']
      )

  def similarities(self, user):
      users = self.remove_user(user)
      return {cur_user: self.similarity_distance(self.common_movies(user, cur_user)) for cur_user in users}

  def other_movies_of_user(self, user_first, user_second):
      return {movieId: rating for movieId, rating in self.user_ratings[user_second].items()
                  if movieId not in self.user_ratings[user_first]}

  def other_movies(self, user):
      users = self.remove_user(user)
      return {cur_user: self.other_movies_of_user(user, cur_user) for cur_user in users}

  def recommendations(self, user):
      similarities = self.similarities(user)
      other_movies = self.other_movies(user)

      recommendations = dict()
      total_similarities = dict()
      for cur_user, movies in other_movies.items():
          for movieId, rating in movies.items():
              recommendations.setdefault(movieId, 0)
              recommendations[movieId] += similarities[cur_user] * rating
              total_similarities.setdefault(movieId, 0)
              total_similarities[movieId] += similarities[cur_user]
      recommendations = {movieId: rating / total_similarities[movieId]
                        for movieId, rating in recommendations.items()}
      ans = dict(sorted(recommendations.items(), key=lambda x: x[1], reverse=True))
      return [{'movieId': key, 'rating': value, 'title': self.movies[key]} for key, value in ans.items()]

Выделим нужные столбцы

In [ ]:
data_df = df[['userId', 'movieId', 'rating', 'title', 'overview']]

Для процесса обучения возьмем 500 пользователей, давшими наибольшое количество оценок. На основе полученного датасета будем обучать наши данные и делать рекоммендации. Также выберем пользователя, давшего наибольшее число оценок

In [ ]:
top_500_users = data_df['userId'].value_counts().head(500).index.values
user = top_500_users[0]
ratings_count = len(data_df.query(f'userId == {user}'))
print(f'Пользователь давший наибольшее число оценок - {user}')
print(f'Количество оценок пользователя: {ratings_count}')
df_5 = data_df[data_df['userId'].isin(top_500_users)].reset_index(drop=True)

Пользователь давший наибольшее число оценок - 45811
Количество оценок пользователя: 3521


Составим рекомендацию из 30 фильмов для просмотра данному пользователю, которые алгоритм предсказал как те, в которых пользователь поставит более высокую оценку:

In [ ]:
collab = Collaborative(df_5)

In [ ]:
recommend = collab.recommendations(user)

In [ ]:
top_30_df = pd.DataFrame(recommend).head(30)

In [ ]:
top_30_df

,movieId,rating,title
0,170689,5.000000,Möbius
1,129360,5.000000,"Usain Bolt, La Légende"
2,164278,5.000000,Harvey
3,74406,5.000000,Queen: Days of Our Lives
4,74491,5.000000,Confessions of an Ugly Stepsister
5,140465,5.000000,Prohibition
6,123109,5.000000,No One Lives
7,68590,5.000000,Legacy of Rage
8,168478,5.000000,Mermaid
9,132912,5.000000,The Price of Sex


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
class Quality:

  def __init__(self, df):
    self.col_filt = Collaborative(df)
    self.df = df

  def evaluate(self, userId, cv, functions: list):
    user_df = self.df.query(f'userId == {userId}')
    metrics = {}
    for function in functions:
      metrics.setdefault(function.__name__, [])
    for train_index, test_index in cv.split(user_df):
        df_test = user_df.iloc[test_index]
        df_train = self.df[~((self.df['userId'].isin(df_test['userId'])) & (self.df['movieId'].isin(df_test['movieId'])))]
        recommendations_df = self.pred_recommendations(user, df_train, df_test)
        for function in functions:
          metric = function(recommendations_df['rating'], recommendations_df['pred'])
          metrics[function.__name__].append(metric)
    return metrics

  def pred_recommendations(self, user, df_train, df_test):
    user_df = self.df.query(f'userId == {user}')
    col_filt = Collaborative(df_train)
    recommendations = col_filt.recommendations(user)
    return pd.merge(
        user_df,
        pd.DataFrame(recommendations)[['movieId', 'rating']].rename(columns={"rating": "pred"}),
        on='movieId'
    )

Оценим качество этой рекомендации для всех фильмов для данного пользователя. Сделаем это с помощью кросс-валидации по 5 фолдам, где в каждом наборе обучающих фолдов будут исключены фильмы, находящиеся в тестовой выборке. В качестве метрики измерения качества рекомендаций возьмем MSE, RMSE, MAE и NDCG@k.

In [ ]:
def ndcg(rel_true, rel_pred, p=5):
    rel_true = np.sort(rel_true)[::-1]
    p = min(len(rel_true), min(len(rel_pred), p))
    discount = 1 / (np.log2(np.arange(p) + 2))
    idcg = np.sum(rel_true[:p] * discount)
    dcg = np.sum(rel_pred[:p] * discount)
    return dcg / idcg

In [ ]:
quality = Quality(df_5)
k = 5
functions_list = [mean_squared_error, mean_absolute_error, ndcg]
metrics = quality.evaluate(user, cv=KFold(n_splits=k), functions=functions_list)

In [ ]:
print(f"MSE (Mean Squared Error): {np.mean(metrics['mean_squared_error'])} \n (усредненное значение на {k} фолдах, чем ниже, тем лучше) \n")
print(f"RMSE (Root Mean Squared Error): {np.mean(np.sqrt(metrics['mean_squared_error']))} \n (усредненное значение на {k} фолдах, чем ниже, тем лучше)\n")
print(f"MAE (Mean Absolute Error): {np.mean(metrics['mean_absolute_error'])} \n (усредненное значение на {k} фолдах, чем ниже, тем лучше)\n")
print(f"NDCG@k (Normalized Discounted Cumulative Gain at k): {np.mean(metrics['ndcg'])} \n (усредненное значение на {k} фолдах, все релевантные значения находятся в топе списка, т.е. чем ближе к 1, тем лучше)")

MSE (Mean Squared Error): 0.5691686613119182 
 (усредненное значение на 5 фолдах, чем ниже, тем лучше) 

RMSE (Root Mean Squared Error): 0.7520554331569256 
 (усредненное значение на 5 фолдах, чем ниже, тем лучше)

MAE (Mean Absolute Error): 0.5408191075062373 
 (усредненное значение на 5 фолдах, чем ниже, тем лучше)

NDCG@k (Normalized Discounted Cumulative Gain at k): 0.6551721519330461 
 (усредненное значение на 5 фолдах, все релевантные значения находятся в топе списка, т.е. чем ближе к 1, тем лучше)


Проведем оценку на большем числе пользователей. Возьмем 50 пользователей с наибольшим количеством оценок

In [ ]:
import sys
top_n = 50
top_50_users_with_most_ratings = df_5['userId'].value_counts().head(top_n).index.values
top_50_users_with_most_ratings

array([ 45811,   8659, 179792, 107720, 270123, 229879, 243443, 228291,
       172224,  70648,  98415, 245739, 194690,  98787, 165352,  59554,
        59449, 186059, 243331,  74275, 230417,  41190, 166928,  89258,
        65469, 141589,  89020,  24025, 176920, 255933,  46156, 101276,
        97293, 267772,  40207, 196541,  32984, 251794, 118673,  33940,
       210792, 172359, 258253, 177150, 180711, 164792,  42957, 174414,
       121083, 225396])

И для каждого из них будем считать среднее значение метрик, а затем усредним каждую из них и получим общую среднюю оценку

In [ ]:
mse_values = []
rmse_values = []
mae_values = []
ndcg_values = []
counter = 1
k = 5
functions_list = [mean_squared_error, mean_absolute_error, ndcg]
for user in top_50_users_with_most_ratings:
  print(f"\rProgress {round((counter/top_n)*100)} %",end="")
  sys.stdout.flush()
  metrics = quality.evaluate(user, cv=KFold(n_splits=k), functions=functions_list)
  mse_values.append(np.mean(metrics['mean_squared_error']))
  rmse_values.append(np.mean(np.sqrt(metrics['mean_squared_error'])))
  mae_values.append(np.mean(metrics['mean_absolute_error']))
  ndcg_values.append(np.mean(metrics['ndcg']))
  counter += 1
print(f"\rСреднее значение усредненного на {k} фолдах MSE (Mean Squared Error) для {top_n} пользователей: {np.mean(mse_values)} \n (чем ниже, тем лучше) \n")
print(f"Среднее значение усредненного на {k} фолдах RMSE (Root Mean Squared Error) для {top_n} пользователей: {np.mean(rmse_values)} \n (чем ниже, тем лучше)\n")
print(f"Среднее значение усредненного на {k} фолдах MAE (Mean Absolute Error): {np.mean(mae_values)} \n (чем ниже, тем лучше)\n")
print(f"Среднее значение усредненного на {k} фолдах NDCG@k (Normalized Discounted Cumulative Gain at k): {np.mean(ndcg_values)} \n (все релевантные значения находятся в топе списка, т.е. чем ближе к 1, тем лучше)")

Среднее значение усредненного на 5 фолдах MSE (Mean Squared Error) для 50 пользователей: 0.9367329180696552 
 (чем ниже, тем лучше) 

Среднее значение усредненного на 5 фолдах RMSE (Root Mean Squared Error) для 50 пользователей: 0.8957500691029537 
 (чем ниже, тем лучше)

Среднее значение усредненного на 5 фолдах MAE (Mean Absolute Error): 0.735159962987331 
 (чем ниже, тем лучше)

Среднее значение усредненного на 5 фолдах NDCG@k (Normalized Discounted Cumulative Gain at k): 0.6541768998865612 
 (все релевантные значения находятся в топе списка, т.е. чем ближе к 1, тем лучше)


Как видим из результатов - все не так плохо. В процентном соотношении доля ошибки занимает в среднем от 14 до 20%.

## Задание 6

Используйте метод Term Frequency Inverse Document Frequency (TF-IDF), чтобы отфильтровать фильмы, похожие (используйте для этого косинусное расстояние) на те, которые пользователь высоко оценил.

Оцените качество такой рекомендации.

Возьмем случайного пользователя с id 30159 (он имеет не так много оценок, так что как раз подходит)

In [ ]:
userId = 30159
ratings_count = len(data_df.query(f'userId == {userId}'))
print(f'Количество оценок пользователя {userId}: {ratings_count}')

Количество оценок пользователя 30159: 174


Обработаем данные и соберем для пользователя

In [ ]:
movies_df = metadata[['id', 'overview', 'title']].rename(columns={'id': 'movieId'})
user_movies = data_df.query(f'userId == {userId}')
user_movies_ids = user_movies['movieId']

Создадим функцию для посчета косинусного расстояния и для первой части задания - рассчет k наиболее похожих фильмов

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idf_sim(train, test, k, mode='compare'):
    Tfidf_Vectorizer = TfidfVectorizer(min_df=0)
    vectorized = Tfidf_Vectorizer.fit_transform(train)
    new_entry = Tfidf_Vectorizer.transform(test)

    cosine_similarities = linear_kernel(new_entry, vectorized).flatten()
    if (mode == 'compare'):
      recc_indices = np.argsort(cosine_similarities)[0:k]
      indexes = list(train.index[recc_indices])
      return indexes
    else:
      return cosine_similarities

Для оценки качества фильтрации фильмов будем использовать leave-one-out кросс-валидацию - будем обучать алгоритм TF-IDF на всех фильмах в датасете, кроме одного тестового. На одном таком фильме будем проверять качество рекомендации, находя наиболее похожие фильмы по косинусному расстоянию.

In [ ]:
from sklearn.model_selection import LeaveOneOut
def run_compare(movies_ids):
  movies_cv_results = {}
  cnt = 0
  top_k = 15
  for train_index, test_index in LeaveOneOut().split(movies_ids):
      cnt += 1
      print(f"\rProgress {(cnt/len(movies_ids))*100} %",end="")
      sys.stdout.flush()
      movie_test = int(movies_ids.iloc[list(test_index)[0]])
      movies_df_train = movies_df[~movies_df['movieId'] != movie_test]
      movie_df_test = movies_df[movies_df['movieId'] == movie_test]
      movies_cv_results[movie_test] = tf_idf_sim(movies_df_train['overview'], movie_df_test['overview'], top_k)
  return movies_cv_results

Progress 100.0 %

In [ ]:
results = run_compare(user_movies_ids)

Мы получили id наиболее похожих фильмов для каждого из фильмов пользователя. Выведем фильмы, похожие на фильм с id 527

In [ ]:
movie = 527
recommendations = list(results[movie])
print(f'Наиболее похожие фильмы (топ {top_k}) на фильм с id равным {movie}:\n')
movies_df.loc[recommendations]

Наиболее похожие фильмы (топ 15) на фильм с id равным 527:



,movieId,overview,title
17095,38944,"Mitä tapahtuu, kun yhdessä miehessä asuu kaksi...",The Leaning Tower
29138,82401,A teen is visited by aliens after he broadcast...,Can of Worms
33228,277967,A thriller crime comedy directed by Wolfgang M...,Life Eternal
43663,322148,Directed by Slavko Spionjak.,Caedes
33218,354667,A 2015 Kannada mystery-thriller movie.,RangiTaranga
21107,160085,An employee at a professional separation agenc...,Schlussmacher
21172,128276,A PC becomes a vigilante after a head trauma.,May I Kill U?
10956,1416,An overzealous soldier mentally deteriorates a...,The Coast Guard
40518,284564,ROB ZOMBIE'S S P O O K H A U S 31,31
33049,245698,American chess champion Bobby Fischer prepares...,Pawn Sacrifice


Однако следуя заданию нам нужны только наиболее оценненые пользователем фильмы. Выберем те фильмы пользователя, где он поставил оценку 5

In [ ]:
user_top_movies = user_movies.sort_values(by='rating', ascending=False).loc[user_movies['rating'] == 5.0]['movieId'].values.tolist()

Составим датасет похожести для полученных результатов

In [ ]:
lstu = []
for item in user_top_movies:
  lstu.append([item, movies_cv_results[item]])
pd.DataFrame(lstu, columns=['movie', 'similar'])

,movie,similar
0,68954,"[22392, 21074, 21067, 2484, 21039, 21017, 2100..."
1,1921,"[17978, 32513, 27361, 10212, 43533, 40516, 187..."
2,541,"[29298, 15448, 33016, 15432, 22745, 22749, 227..."
3,1282,"[34515, 30129, 24131, 24133, 7215, 1232, 14108..."
4,1682,"[16490, 42563, 13771, 24469, 24507, 33738, 425..."
5,2291,"[11580, 24653, 1489, 24507, 7111, 24469, 1476,..."
6,527,"[17095, 29138, 33228, 43663, 33218, 21107, 211..."
7,899,"[8058, 23886, 12375, 19065, 29272, 23917, 4456..."
8,6970,"[37426, 34629, 31855, 11833, 43752, 38775, 235..."
9,318,"[27361, 14266, 908, 6414, 42515, 34763, 3007, ..."


Для оценки качетсва будет считать процент попадания в заданный интервал (уровень выбран как половина от среднего значения) косинусного расстояния между случайными фильмами и фильмами из рекомендации для разных пользователей. Затем усредним и получим процент попадания (hit rate)

In [ ]:
def run_compare_analyze(movies_ids):
  top_k = 15
  cossims = []
  for train_index, test_index in LeaveOneOut().split(movies_ids):
    sys.stdout.flush()
    movie_test = int(movies_ids.iloc[list(test_index)[0]])
    movies_df_train = movies_df[~movies_df['movieId'] != movie_test]
    movie_df_test = movies_df[movies_df['movieId'] == movie_test]
    cossim = tf_idf_sim(movies_df_train['overview'], movie_df_test['overview'], top_k, mode='not')
    i = 0
    for item in cossim:
      if item > 0.015:
        i += 1
    cossims.append(100 * i/len(cossim))
  return np.mean(cossims)

In [ ]:
def count_hit(users, rates, test_n):
  for i in range(0, len(users)):
    print(f"\rProgress {(i+1/len(users))*100} %",end="")
    user = users[i]
    comp_movie_ids = data_df.query(f'userId == {user}')['movieId'][:test_n]
    rates.append(run_compare_analyze(comp_movie_ids))
  return np.mean(rates)

Выполним для 20 пользователей с 20 фильмами каждый

In [ ]:
import random
users = random.sample(list(top_500_users), 20)
rates = []
hit = count_hit(users, rates, 20)
print('\r', hit, '%')

 70.64776031921369 %


Как видим получили процент попадания 70, что очень неплохо. Таким образом можно составлять рекомендации по многим видам данных, тк метод весьма эффективен (при большем количестве итераций обьективность будет выше, сведения об эффективности приводятся в текущем контексте)